### Simple Graph

In [24]:
import tensorflow as tf
x=tf.Variable(3.0, name="x", dtype=tf.float32)
y=tf.Variable(4.0, name="y", dtype=tf.float32)

f = x*x*y+y+2

- Above statement only defines tensorflow graph it doesn't yet initialize variables or do calculations
- x,y and f are nodes and tensorflow graph. Which are automatically added to default graph

In [3]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
sess.close()
result

42.0

### Using "with" Block

In [9]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
result

42

### Initializing Global Variables At Once

In [10]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()
result

42

### Interactive Session
- To do experiment with one session, need to call close after done

In [11]:
in_sess = tf.InteractiveSession() # Crete and set session as default
init.run()
result = f.eval()
in_sess.close()
result


42

### Gradient Descent Using Tensorflow

In [1]:
from sklearn.datasets import fetch_california_housing
from pandas import DataFrame
import numpy as np
from sklearn.preprocessing import StandardScaler


housing_data = fetch_california_housing()
m,n = housing_data.data.shape
scaled_data = StandardScaler().fit_transform(housing_data.data)
housing_data_df = DataFrame(scaled_data, columns=housing_data.feature_names)
housing_data_df["bais"] = np.ones(m)
columns = ["bais"] + list(housing_data_df.columns[:-1])
housing_data_df = housing_data_df[columns]
housing_data_df.head()

,bais,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,1.0,2.344766,0.982143,0.628559,-0.153758,-0.974429,-0.049597,1.052548,-1.327835
1,1.0,2.332238,-0.607019,0.327041,-0.263336,0.861439,-0.092512,1.043185,-1.322844
2,1.0,1.782699,1.856182,1.155620,-0.049016,-0.820777,-0.025843,1.038503,-1.332827
3,1.0,0.932968,1.856182,0.156966,-0.049833,-0.766028,-0.050329,1.038503,-1.337818
4,1.0,-0.012881,1.856182,0.344711,-0.032906,-0.759847,-0.085616,1.038503,-1.337818


In [101]:
import tensorflow as tf

lr = 0.01

X = tf.constant(housing_data_df.values, dtype=tf.float32, name="X")
y = tf.constant(housing_data.target.reshape(-1,1), dtype=tf.float32, name="y")

theta = tf.Variable(tf.random_uniform([n+1,1], -1.0,1.0,dtype=
                                     tf.float32), dtype=tf.float32, name="theta")
preds_op = tf.matmul(X,theta,name="predictions")
error_op = preds_op - y
grad_op = (2/m) * tf.matmul(tf.transpose(X), error_op, name="gradients")
training_op = tf.assign(theta, theta-lr*grad_op)

mse_op = tf.reduce_mean(tf.square(error_op), name="mse")
init_op = tf.global_variables_initializer()

In [122]:
def perform_gradient_descent(training_op, n_epoch=1000):
    with tf.Session() as sess:
        ## The line below is added to avoid error 
        ## "FailedPreconditionError: Attempting to use uninitialized value theta_23"
        ## when using MomentumOptimizer
        tf.initialize_all_variables().run()
        sess.run(init_op)
        theta.eval()
        for epoch in range(n_epoch):
            sess.run(training_op)
            if epoch%100==0:
                print(mse_op.eval())
        best_theta = theta.eval()
        print(best_theta)

In [96]:
perform_gradient_descent(training_op)

12.42785
0.98547685
0.72801405
0.6683158
0.62809056
0.5991696
0.57831275
0.56326824
0.552416
0.5445879
[[ 2.0685523 ]
 [ 0.75455   ]
 [ 0.15247895]
 [-0.03428616]
 [ 0.07515995]
 [ 0.00874044]
 [-0.04057861]
 [-0.72451144]
 [-0.68210423]]


### Using Auto Diff
- Auto diff calculates the gradient numerically with respect to given variable

In [108]:
auto_diff_grad = tf.gradients(mse_op,[theta])[0]
auto_diff_train_op = tf.assign(theta, theta-lr*auto_diff_grad)

perform_gradient_descent(auto_diff_train_op)

6.668069
0.7467046
0.63447845
0.6107754
0.5934411
0.5798341
0.56907713
0.560532
0.55371314
0.548249
[[ 2.0685525 ]
 [ 0.9469755 ]
 [ 0.16062963]
 [-0.45007995]
 [ 0.44195756]
 [ 0.00920807]
 [-0.04537104]
 [-0.484403  ]
 [-0.46691895]]


### Using Optimizer
- Optimizer automatically performs gradient descent on cost function operator
- It automatically detects the variables in the cost function and change it to reduce the cost function

#### Gradient Descent Optimiser

In [107]:
tf.train.GradientDescentOptimizer(learning_rate=lr)
gd_optimiser_train_op = optimizer.minimize(mse_op)
perform_gradient_descent(gd_optimiser_train_op)

7.648159
0.7675881
0.6371839
0.60500616
0.5829837
0.5670564
0.55549943
0.5471045
0.5409999
0.5365559
[[ 2.0685525 ]
 [ 0.8178526 ]
 [ 0.15121251]
 [-0.17845586]
 [ 0.2051815 ]
 [ 0.00745886]
 [-0.04182141]
 [-0.67430985]
 [-0.64045894]]


#### Momentum Optimizer

In [124]:
m_optimizer = tf.train.MomentumOptimizer(learning_rate=lr, momentum=0.75)
momentum_optimizer_train_op = m_optimizer.minimize(mse_op)
perform_gradient_descent(momentum_optimizer_train_op)

3.8320448
0.5571656
0.5362444
0.5290273
0.5263092
0.52520233
0.5247241
0.5245088
0.5244095
0.5243629
[[ 2.0685587 ]
 [ 0.83449966]
 [ 0.11978285]
 [-0.27453005]
 [ 0.3130455 ]
 [-0.00419122]
 [-0.03951756]
 [-0.8877857 ]
 [-0.8589985 ]]


### Minibatch Gradient Descent

In [17]:

def fetch_minibatch(batch_size):
    n_batches =  m//batch_size
    rand_index = np.random.permutation(m)
    X = housing_data_df.values
    for batch in range(n_batches):
        cur_index = rand_index[batch*batch_size:(batch+1)*batch_size]
        y = housing_data.target.reshape(-1,1)[cur_index, :]
        yield (X[cur_index,:], y)
    
    
    

In [22]:
def perform_minibatch_gd(n_epoch=500):
    lr=0.001
    batch_size=500
    ## placeholder is used to send parameters when running the operations
    X_mini = tf.placeholder(dtype=tf.float32, shape=(None,  n+1), name="X")
    y_mini = tf.placeholder(dtype=tf.float32, shape=(None,  1), name="y")

    theta = tf.Variable(tf.random_uniform([n+1,1], -1.0,1.0,dtype=
                                         tf.float32), dtype=tf.float32, name="theta")
    preds_op = tf.matmul(X_mini,theta,name="predictions")
    error_op = preds_op - y_mini
    grad_op = (2/batch_size) * tf.matmul(tf.transpose(X_mini), error_op, name="gradients")
    training_op = tf.assign(theta, theta-lr*grad_op)
    
    mse_mini = tf.reduce_mean(tf.square(error_op))
    
    
    
    with tf.Session() as sess:
        init_op = tf.global_variables_initializer()
        sess.run(init_op)
        for epoch in range(n_epoch):
            for x,y in fetch_minibatch(batch_size):
                sess.run(training_op, feed_dict={X_mini:x, y_mini:y})
            
            if epoch%100==0:
                print(mse_mini.eval(feed_dict={X_mini:x, y_mini:y}))
        best_theta = theta.eval()
    
    return best_theta
                

In [23]:
perform_minibatch_gd()

6.454883
0.644192
0.4661135
0.6776871
0.506116


array([[ 2.0685449e+00],
       [ 8.2534415e-01],
       [ 1.2637660e-01],
       [-2.4194574e-01],
       [ 2.7881083e-01],
       [-1.7925644e-03],
       [-3.9789394e-02],
       [-8.4902823e-01],
       [-8.1848347e-01]], dtype=float32)